# Detect Clusters of Vegetation Areas
This notebook retrieves the centroids, using center of gravity, and the cluster points of all vegetation areas with a NDVI > 0.9

In [3]:
# Import necessary libraries
import pickle
import requests
import cv2
import pandas as pd
import rasterio
import numpy as np
import subprocess

In [ ]:
## Parameters

In [4]:
# Retrieve an image using rasdaman
service_endpoint = "https://ows.rasdaman.org/rasdaman/ows"
# Parameters
threshold = 0.9
output_format = "image/tiff"
output_file = "query_result.tiff"